In [ ]:
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
import optuna
from utils import run_study



In [ ]:
# CONFIG

kfolds = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)



In [ ]:

df = pd.read_csv('data/data_preprocessed/qscore.csv')



In [ ]:

# WITH MISSING DATA

train = df[df["year"] != "2018/19"].drop(["year"], axis=1)
X_train = train.drop(["hip_replacement_post_op_q_score_bin"], axis=1)
y_train = train["hip_replacement_post_op_q_score_bin"]

test = df[df["year"] == "2018/19"].drop(["year"], axis=1)
X_test = test.drop(["hip_replacement_post_op_q_score_bin"], axis=1)
y_test = test["hip_replacement_post_op_q_score_bin"]


In [ ]:
# WITHOUT MISSING
train_nm = train.dropna()
X_train_nm = train_nm.drop(["hip_replacement_post_op_q_score_bin"], axis=1)
y_train_nm = train_nm["hip_replacement_post_op_q_score_bin"]

test_nm = test.dropna()
X_test_nm = test_nm.drop(["hip_replacement_post_op_q_score_bin"], axis=1)
y_test_nm = test_nm["hip_replacement_post_op_q_score_bin"]


In [ ]:

# skal have kørt pipelines ind i obj_func
# den smider nogle fejl
# har ikke weightet, one_hottet eller noget andet ...

# jeg kan lave split ds funktion så jeg splitter inde i objektive .. kan teste om missing med impute eller ej virker


def obj_logr(trial):
    logreg = LogisticRegression(
        solver="saga",
        l1_ratio=trial.suggest_float("l1_ratio", 0, 1),
        C=trial.suggest_loguniform("C", 1e-6, 1))

    cv_score = cross_val_score(logreg, X_train_nm, y_train_nm,
                               scoring="roc_auc", cv=kfolds, n_jobs=-1)
    return np.mean(cv_score)


In [ ]:


study_logr = run_study(obj_logr, n_trials=10, study_name="logr")



In [ ]:

logr = logreg = LogisticRegression(
    solver="saga",
    l1_ratio=0.83,
    C=0.05)

logr.fit(X_train_nm, y_train_nm)


In [ ]:
probas = logr.predict_proba(X_test_nm)
roc_auc_score(y_test_nm, probas[:, 1])

In [ ]:

# SKAL ÆNDRE NAMES TIL AT VÆRE SAMME SOM ARG SÅ JEG KAN BRUGE DICT FRA STUDY


def obj_lgbm(trial):
    lgbm = LGBMClassifier(
        learning_rate=0.1,
        max_depth=trial.suggest_int('max_depth', 1, 15),  # usually 3-12 -> tune!
        num_iteration=trial.suggest_int("num_iter", 1, 2000),  # 1 - inf -> tune
        # tune typical 255 (can be > 4000) -> tune!
        num_leaves=trial.suggest_int("num_leaves", 1, 2000),
        min_data_in_leaf=trial.suggest_int("data_in_leaf", 1, 50),  # 1- 50 -> Tune!
        # prop of rows to sample # typical 0.7 (reducing may improve generalize better?)
        sub_row=trial.suggest_int("sub_row", 70, 100) / 100,
        # same as sub_row but for cols
        sub_feature=trial.suggest_int("sub_cols", 70, 100) / 100,
        use_missing=True,
        is_unbalanced=True
    )
    cv_score = cross_val_score(lgbm, X_train, y_train,
                               scoring="roc_auc", cv=kfolds, n_jobs=-1)
    return np.mean(cv_score)



In [ ]:
best_lgbm = study.best_params

In [ ]:
lgbm = LGBMClassifier(
    learning_rate=0.1,
    max_depth=14,
    num_iteration=388,
    num_leaves=1559,
    min_data_in_leaf=33,
    sub_row=0.99,
    sub_feature=0.93,
    use_missing=True,
    is_unbalanced=True
)

#lgbm.fit(X_train, y_train)
repeat_kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
cv_score = cross_val_score(lgbm, X_train, y_train,
                           scoring="roc_auc", cv=repeat_kfold, n_jobs=-1)



In [ ]:
probas = lgbm.predict_proba(X_test)
roc_auc_score(y_test, probas[:, 1]),


In [ ]:
preds = lgbm.predict(X_test)
f1_score(y_test, preds)
